In [42]:
import numpy as np
import pandas as pd
import math

In [43]:
modelX= pd.read_excel("output/predicted_trajectoriesX.xlsx")
modelY= pd.read_excel("output/predicted_trajectoriesY.xlsx")
modelZ= pd.read_excel("output/predicted_trajectoriesZ.xlsx")

**Every measurement is in mm**

In [57]:
def getTimeFromZ(contactHeight = 1300):
    CONTACT_Z = contactHeight
    MAX_HEIGHT= modelZ["LocationZ"].max()
    passedMaxHeight = False


    #using for loop is important as the sequence of these numbers matter, need to start at the top which is at time=0s
    for i in range(modelZ["LocationZ"].size):
        if not passedMaxHeight and abs(modelZ["LocationZ"][i] - MAX_HEIGHT) < 1e-9 :
            passedMaxHeight=True
        if passedMaxHeight:
            if modelZ["LocationZ"][i] == CONTACT_Z:
                #print(f'Time is {modelZ["time"][i]}')
                return modelZ["time"][i]
            elif modelZ["LocationZ"][i] < CONTACT_Z:
                heightBelow = modelZ["LocationZ"][i]
                heightAbove = modelZ["LocationZ"][i-1]
                result = (((modelZ["time"][i]- modelZ["time"][i-1])*(CONTACT_Z - heightAbove))/(heightBelow-heightAbove)) + (modelZ["time"][i-1])
                #print(f"Time is {result}")
                return result
        
        
def getXAtTime(timeOfContact):
    
    for i in range(modelX["LocationX"].size):
        if modelX["time"][i] == timeOfContact:
           #print(f'Point of contact X is {modelX["LocationX"][i]}')
           return modelX["LocationX"][i]
        elif modelX["time"][i] > timeOfContact:
           timeBefore = modelX["time"][i-1]
           timeAfter = modelX["time"][i]
           result = (((modelX["LocationX"][i]- modelX["LocationX"][i-1])*(timeOfContact-timeBefore))/(timeAfter-timeBefore)) + modelX["LocationX"][i-1]
           #print(f"Point of contact X is {result}")
           return result
    
def getYAtTime(timeOfContact):
    
    for i in range(modelY["LocationY"].size):
        if modelX["time"][i] == timeOfContact:
           #print(f'Point of contact Y is {modelY["LocationY"][i]}')
           return modelY["LocationY"][i]
        elif modelY["time"][i] > timeOfContact:
           timeBefore = modelY["time"][i-1]
           timeAfter = modelY["time"][i]
           result = (((modelY["LocationY"][i]- modelY["LocationY"][i-1])*(timeOfContact-timeBefore))/(timeAfter-timeBefore)) + modelY["LocationY"][i-1]
           #print(f'Point of contact Y is {result}')
           return result
    


**Driver code**

In [60]:
CONTACT_Z = 1300
LENGTH_OF_RACKET = 630
HEIGHT_OF_ROBOT = 500
targetPoint = (13400,5640,0.0) #put 3D even though we assume Z-coordinate of landing point to forever be 0
DURATION_OF_SWING = 0.0

timeOfContact = getTimeFromZ(CONTACT_Z)
pointOfContact= getXAtTime(timeOfContact), getYAtTime(timeOfContact), CONTACT_Z

launchAngle= math.sin((CONTACT_Z-HEIGHT_OF_ROBOT)/(LENGTH_OF_RACKET))
launchDirection= math.tan((targetPoint[1]-pointOfContact[1])/(targetPoint[0]-pointOfContact[0]))

ROBOT_X = (LENGTH_OF_RACKET* math.cos(launchAngle) * math.cos(launchDirection)) + pointOfContact[0]
ROBOT_Y = (LENGTH_OF_RACKET* math.cos(launchAngle) * math.sin(launchDirection)) + pointOfContact[1]

robotPoint= (ROBOT_X,ROBOT_Y,0.0)
timeStartSwing = timeOfContact - DURATION_OF_SWING

print(f"Point of contact is {pointOfContact}")
print(f"Robot coordinates are {robotPoint}")
print(f"Launch Direction is {launchDirection}")
print(f"Time to start swinging is {timeStartSwing}")


Point of contact is (8241.691201050804, -8.095726690241776e-08, 1300)
Robot coordinates are (8604.337344594634, -29.767850732257475, 0.0)
Launch Direction is -0.08190149522284644
Time to start swining is 0.6999655121213386


Just some thoughts.... 
since the type of shot we want to make is directyly affected by the swing angle beta, shouldnt the proper flow be:

1. use to determine the best type of shot that wins the game
2. from type of shot we determine the swing angle beta
3. From the swing angle we then calculate what height we should hit, ie what CONTACT_Z should be

since we are skipping the AI part and directly fixing the swing angle, we want to hit at a specific swing angle so we cannot fix the CONTACT_Z

The problem is we cant get time out of the AI trajectory data model. Since we trained it to output the location coordinates, it can only output the entire trajectory (up to the time we want it to) and we can only cut the trajectory data at the height we want to hit (either by fixing it to a constant or fixing the swing angle and using that to calcuate instead).

Idea: only tell the robot where to move in terms of X and Y, but instead of telling fixing its exact hit/swing angle, tell it what time to hit and let it have the full range of motion of swing, that way it has higher chance of hitting the shuttlecock on its way down. We only need to calcuate the swing angle to determine X and Y so the robot knows where to go, but we dont actually have to fix the swing angle.

Future: look into ways of letting it train and output 3 axis coordinates at once, and for faster run speed, dont save it into excel file

In [59]:
modelX.head(30)

,time,LocationX
0,0.000000,1831.591158
1,0.033333,2900.686774
2,0.066667,3895.740065
3,0.100000,4359.462889
4,0.133333,4908.677389
5,0.166667,5486.642320
6,0.200000,5700.138038
7,0.233333,6001.457961
8,0.266667,6347.371853
9,0.300000,6511.845428
